In [34]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

flow = pd.read_csv('CurrentWater.csv')
rain = pd.read_csv('CurrentPrecip.csv',index_col=0)
rainforecast = pd.read_csv('WeatherForecast.csv',index_col=0)
reference = pd.read_csv('RiverReferenceTable.csv')

flow = flow.set_index('Name').join(reference[['USGS Name','Name']].set_index('USGS Name')).reset_index()
flow['CFS'] = flow.Value
flow['DateTime'] = pd.to_datetime(flow.DateTime)
flow['Date'] = flow.DateTime.dt.date
flow = flow[['Date','Name','CFS']].groupby(['Date','Name']).mean().reset_index()
flow['Date_Name'] = flow.Date.astype(str)+'_'+flow.Name

rain = rain.drop_duplicates()
rain['Date_Name'] = rain.Date+'_'+rain.Name

df = flow.set_index('Date_Name').join(rain[['Date_Name','Precip']].set_index('Date_Name'))
df.head()

,Date,Name,CFS,Precip
Date_Name,,,,
2021-12-04_East Fork Carson,2021-12-04,East Fork Carson,85.455000,NaN
2021-12-04_East Walker,2021-12-04,East Walker,23.400000,NaN
2021-12-04_Eel River Fort Seward,2021-12-04,Eel River Fort Seward,558.750000,NaN
2021-12-04_Eel River Scotia,2021-12-04,Eel River Scotia,911.500000,NaN
2021-12-04_Hat Creek,2021-12-04,Hat Creek,56.240625,NaN


In [40]:
df.loc[df.Name == 'Van Duzen River Bridgeville'].tail(50)

,Date,Name,CFS,Precip
Date_Name,,,,
2022-01-05_Van Duzen River Bridgeville,2022-01-05,Van Duzen River Bridgeville,5537.625000,NaN
2022-01-06_Van Duzen River Bridgeville,2022-01-06,Van Duzen River Bridgeville,3511.484375,NaN
2022-01-07_Van Duzen River Bridgeville,2022-01-07,Van Duzen River Bridgeville,3994.947917,NaN
2022-01-08_Van Duzen River Bridgeville,2022-01-08,Van Duzen River Bridgeville,2859.270833,NaN
2022-01-09_Van Duzen River Bridgeville,2022-01-09,Van Duzen River Bridgeville,1856.510417,NaN
2022-01-10_Van Duzen River Bridgeville,2022-01-10,Van Duzen River Bridgeville,1423.385417,NaN
2022-01-11_Van Duzen River Bridgeville,2022-01-11,Van Duzen River Bridgeville,1212.031250,NaN
2022-01-12_Van Duzen River Bridgeville,2022-01-12,Van Duzen River Bridgeville,1095.000000,NaN
2022-01-13_Van Duzen River Bridgeville,2022-01-13,Van Duzen River Bridgeville,1011.963542,NaN


In [37]:
df.loc[df.Precip.isnull()]

,Date,Name,CFS,Precip
Date_Name,,,,
2021-12-04_East Fork Carson,2021-12-04,East Fork Carson,85.455000,NaN
2021-12-04_East Walker,2021-12-04,East Walker,23.400000,NaN
2021-12-04_Eel River Fort Seward,2021-12-04,Eel River Fort Seward,558.750000,NaN
2021-12-04_Eel River Scotia,2021-12-04,Eel River Scotia,911.500000,NaN
2021-12-04_Hat Creek,2021-12-04,Hat Creek,56.240625,NaN
...,...,...,...,...
2022-03-13_Truckee River Farad,2022-03-13,Truckee River Farad,419.406250,NaN
2022-03-13_Umpqua River Elkton,2022-03-13,Umpqua River Elkton,3531.093750,NaN
2022-03-13_Van Duzen River Bridgeville,2022-03-13,Van Duzen River Bridgeville,98.821875,NaN
